In [18]:
#pip install selenium
#pip install webdriver_manager
#pip install BeautifulSoup4
#pip install bs4

In [3]:
# from selenium import webdriver
# from selenium.webdriver.common.keys import Keys # enter 치는 명령어
# import pandas as pd 
# from selenium.webdriver.common.by import By
# from selenium.webdriver.chrome.service import Service

# from webdriver_manager.chrome import ChromeDriverManager



from selenium import webdriver
import time
from bs4 import BeautifulSoup
import re

In [68]:
page = requests.get('http://symbol.ksaac.or.kr/searchsymbols/searchbycategory?cago=23')
soup = bs(page.text, "html.parser")

elements_title = soup.select('.category-l1-name')


In [1]:
import requests
from bs4 import BeautifulSoup as bs

#해당 카테고리가 있는 주소를 넣으면 그 카테고리에 있는 토큰과 이미지를 가져오는 함수

def get_text(url):
    page = requests.get(url)
    soup = bs(page.text, "html.parser")

    elements_text = soup.select('.s-expression')
    #elements_image = soup.select('.s-img')


    li = []

    for i in range(len(elements_text)):
        try:
            li.append(elements_text[i].text)
        except:
            print(f"{i}번째 데이터에 오류가 있습니다")

    return li


def get_image(url):
    page = requests.get(url)
    soup = bs(page.text, "html.parser")

    #elements_text = soup.select('.s-expression')
    elements_image = soup.select('.s-img')


    li = []

    for i in range(len(elements_image)):
        try:
            li.append(elements_image[i]['src'])
        except:
            print(f"{i}번째 데이터에 오류가 있습니다")

    return li

In [70]:
page = requests.get('http://symbol.ksaac.or.kr/searchsymbols/searchbycategory?cago=23')
soup = bs(page.text, "html.parser")

elements_title = soup.select('.category-l1-name')


# csv_text = []
# csv_image = []

# 모든 카테고리를 더해서 하나의 데이터 프레임을 만듬

for i in range(25):
    if i < 23:
        url = f"http://symbol.ksaac.or.kr/searchsymbols/searchbycategory?cago={i+2}"
    else:
        i = i + 133
        url = f"http://symbol.ksaac.or.kr/searchsymbols/searchbycategory?cago={i}"

    if i == 0:
        csv_text = get_text(url)
        csv_image = get_image(url)
    else:
        csv_text = csv_text + get_text(url)
        csv_image = csv_image + get_image(url)
        
    
    

In [ ]:
df.to_csv('aac.csv',index = False,encoding="utf-8-sig")

In [72]:
len(csv_text)

10359

In [73]:
len(csv_image)

10359

In [75]:
import pandas as pd

df = pd.DataFrame({'text':csv_text,
            'image':csv_image})

In [2]:
# 음식 카테고리만 따로 추출함


url = "http://symbol.ksaac.or.kr/searchsymbols/searchbycategory?cago=3"

li1 = get_text(url)
li2 = get_image(url)

import pandas as pd

df_food = pd.DataFrame({'text':li1,
            'image':li2})

In [4]:
df_food['카테고리1'] = '음식'

In [5]:
df_food.head()

,text,image,카테고리1
0,간장에찍어요,http://220.69.171.28:8080/Symbol/Symbol/n2_eum...,음식
1,갈비탕,http://220.69.171.28:8080/Symbol/Symbol/n2_eum...,음식
2,고추잡채,http://220.69.171.28:8080/Symbol/Symbol/n2_eum...,음식
3,고추장넣어요,http://220.69.171.28:8080/Symbol/Symbol/n2_eum...,음식
4,고추장주세요,http://220.69.171.28:8080/Symbol/Symbol/n2_eum...,음식


In [17]:
#종성 뽑아오는 함수
from jamo import h2j, j2hcj

def extract_consonant(text):
    text = j2hcj(h2j(text))[-1]

    return text

In [15]:
extract_consonant("토마토")

'ㅗ'

In [43]:
from jamo import h2j, j2hcj

def noun(df):
    idx = []
    for i in range(df.shape[0]):
        temp = df.iloc[i,0]
        text = extract_consonant(temp)
        if text[-1] == 'ㅁ' and len(temp) >= 3 :
            idx.append(i)

    return pd.DataFrame(set(df.iloc[idx,0]))

In [44]:
df_food_noun = noun(df_food)

In [38]:
df_food_noun.shape[0]

19

In [48]:
df_food_noun.iloc[0][0] + "를 좋아애효"

'땅콩잼를 좋아애효'

In [49]:
#음식과 관련된 동사형
#먹고 싶어요, 더 주세요, ~을 좋아해요
#마지막에 종성이 있으면 ~를 종성이 없으면 ~을 붙혀서 동사와 붙혀서 명사로 만들자

list = ['ㅏ','ㅐ','ㅑ','ㅒ','ㅓ','ㅔ','ㅕ','ㅖ','ㅗ','ㅘ','ㅙ','ㅚ','ㅛ','ㅜ','ㅝ','ㅞ','ㅟ','ㅠ','ㅡ','ㅢ','ㅣ']


verb = ['먹고 싶어요','더 주세요','좋아해요']

script_list = []


for i in range(len(verb)):
    for j in range(df_food_noun.shape[0]):
        temp = df_food_noun.iloc[j][0]
        
        if extract_consonant(temp) in list:
            script_list.append(temp+"를 "+verb[i])
        else:
            script_list.append(temp+"을 "+verb[i])




In [50]:
script_list

['땅콩잼을 먹고 싶어요',
 '멸치볶음을 먹고 싶어요',
 '제육볶음을 먹고 싶어요',
 '과일통조림을 먹고 싶어요',
 '콘아이스크림을 먹고 싶어요',
 '살구잼을 먹고 싶어요',
 '참기름을 먹고 싶어요',
 '장조림을 먹고 싶어요',
 '콩나물무침을 먹고 싶어요',
 '감자튀김을 먹고 싶어요',
 '포도잼을 먹고 싶어요',
 '참치통조림을 먹고 싶어요',
 '사과잼을 먹고 싶어요',
 '딸기잼을 먹고 싶어요',
 '시금치무침을 먹고 싶어요',
 '어묵볶음을 먹고 싶어요',
 '아이스크림을 먹고 싶어요',
 '갈비찜을 먹고 싶어요',
 '들기름을 먹고 싶어요',
 '땅콩잼을 더 주세요',
 '멸치볶음을 더 주세요',
 '제육볶음을 더 주세요',
 '과일통조림을 더 주세요',
 '콘아이스크림을 더 주세요',
 '살구잼을 더 주세요',
 '참기름을 더 주세요',
 '장조림을 더 주세요',
 '콩나물무침을 더 주세요',
 '감자튀김을 더 주세요',
 '포도잼을 더 주세요',
 '참치통조림을 더 주세요',
 '사과잼을 더 주세요',
 '딸기잼을 더 주세요',
 '시금치무침을 더 주세요',
 '어묵볶음을 더 주세요',
 '아이스크림을 더 주세요',
 '갈비찜을 더 주세요',
 '들기름을 더 주세요',
 '땅콩잼을 좋아해요',
 '멸치볶음을 좋아해요',
 '제육볶음을 좋아해요',
 '과일통조림을 좋아해요',
 '콘아이스크림을 좋아해요',
 '살구잼을 좋아해요',
 '참기름을 좋아해요',
 '장조림을 좋아해요',
 '콩나물무침을 좋아해요',
 '감자튀김을 좋아해요',
 '포도잼을 좋아해요',
 '참치통조림을 좋아해요',
 '사과잼을 좋아해요',
 '딸기잼을 좋아해요',
 '시금치무침을 좋아해요',
 '어묵볶음을 좋아해요',
 '아이스크림을 좋아해요',
 '갈비찜을 좋아해요',
 '들기름을 좋아해요']

In [51]:
len(script_list)

57